# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 5412, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 5412 (delta 42), reused 24 (delta 7), pack-reused 5314 (from 2)
Receiving objects: 100% (5412/5412), 384.32 MiB | 23.14 MiB/s, done.
Resolving deltas: 100% (3088/3088), done.
Updating files: 100% (445/445), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 23.5 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprecated NumPy API, disabl

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithoutKFCV/Hybrid'

np.random.seed(42)

## Import the repository

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'RP3betaHybrid',
    'metric': 'MAP',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_RP3betaHybrid_MAP.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [12]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

with open(f'{K_PATH}/TrainedModels/WithoutKFCV/GraphBased/RP3betaRecommender/Optimizing{config["metric"]}/best_params_RP3beta_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

rp3beta_recommender = RP3betaRecommender(URM_train)
rp3beta_recommender.fit(**best_params)

with open(f'{K_PATH}/TrainedModels/WithoutKFCV/GraphBased/RP3betaICMRecommender/Optimizing{config["metric"]}/best_params_RP3betaICM_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

rp3beta_icm_recommender = RP3betaRecommender(ICM_all.T)
rp3beta_icm_recommender.fit(**best_params)

RP3betaRecommender: Similarity column 38121 (100.0%), 2198.62 column/sec. Elapsed time 17.34 sec
RP3betaRecommender: Similarity column 38121 (100.0%), 752.74 column/sec. Elapsed time 50.64 sec


In [13]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from sklearn.preprocessing import normalize

def objective_function_RP3betaHybrid(optuna_trial):
    
    icm_weight = optuna_trial.suggest_float("icm_weight", 0.0, 1.0)

    # Normalize and combine weight matrices
    normalized_rp3beta = normalize(rp3beta_recommender.W_sparse, norm='l1', axis=1)
    normalized_rp3beta_icm = normalize(rp3beta_icm_recommender.W_sparse, norm='l1', axis=1)
    custom_similarity = normalized_rp3beta + icm_weight * normalized_rp3beta_icm
    
    recommender_instance = ItemKNNCustomSimilarityRecommender(URM_train)
    recommender_instance.fit(custom_similarity)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [14]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_RP3betaHybrid, n_trials=400)

[I 2025-01-02 01:16:36,124] A new study created in RDB with name: hyperparameters_tuning_RP3betaHybrid_MAP


EvaluatorHoldout: Processed 35595 (100.0%) in 31.93 sec. Users per second: 1115


[I 2025-01-02 01:17:08,222] Trial 0 finished with value: 0.044341721349305176 and parameters: {'icm_weight': 0.2641917415710384}. Best is trial 0 with value: 0.044341721349305176.


EvaluatorHoldout: Processed 35595 (100.0%) in 31.76 sec. Users per second: 1121


[I 2025-01-02 01:17:40,108] Trial 1 finished with value: 0.040463805777963976 and parameters: {'icm_weight': 0.6777916987600546}. Best is trial 0 with value: 0.044341721349305176.


...

[I 2025-01-02 04:50:49,587] Trial 397 finished with value: 0.04381854728125131 and parameters: {'icm_weight': 0.41798923294549795}. Best is trial 91 with value: 0.044400295877117.


EvaluatorHoldout: Processed 35595 (100.0%) in 31.92 sec. Users per second: 1115


[I 2025-01-02 04:51:21,651] Trial 398 finished with value: 0.044077159936408365 and parameters: {'icm_weight': 0.19780956316458512}. Best is trial 91 with value: 0.044400295877117.


EvaluatorHoldout: Processed 35595 (100.0%) in 32.00 sec. Users per second: 1112


[I 2025-01-02 04:51:53,800] Trial 399 finished with value: 0.04435931901439548 and parameters: {'icm_weight': 0.27058345117234617}. Best is trial 91 with value: 0.044400295877117.


## Some optuna visualizations on recommender parameters

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [16]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [17]:
with open(f'{K_PATH}/TrainedModels/WithoutKFCV/GraphBased/RP3betaRecommender/Optimizing{config["metric"]}/best_params_RP3beta_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

rp3beta_recommender = RP3betaRecommender(URM_train + URM_validation)
rp3beta_recommender.fit(**best_params)

RP3betaRecommender: Similarity column 38121 (100.0%), 1953.21 column/sec. Elapsed time 19.52 sec


In [18]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    # Normalize and combine weight matrices
    normalized_rp3beta = normalize(rp3beta_recommender.W_sparse, norm='l1', axis=1)
    normalized_rp3beta_icm = normalize(rp3beta_icm_recommender.W_sparse, norm='l1', axis=1)
    custom_similarity = normalized_rp3beta + best_params["icm_weight"] * normalized_rp3beta_icm
    
    recommender_instance = ItemKNNCustomSimilarityRecommender(URM_train + URM_validation)
    recommender_instance.fit(custom_similarity)

# Testing

Create the recommendations for the submission. 

In [19]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_RP3betaHybrid_MAP.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [20]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithoutKFCV/Hybrid/RP3betaHybridRecommender/OptimizingMAP/best_params_RP3betaHybrid_MAP.json' created successfully.


Save the history of the tuned model.

In [21]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithoutKFCV/Hybrid/RP3betaHybridRecommender/OptimizingMAP/history_RP3betaHybrid_MAP.db' created successfully.


Save the best trained model and its submission.

In [22]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithoutKFCV/Hybrid/RP3betaHybridRecommender/OptimizingMAP/Submission/submission_RP3betaHybrid_MAP.csv' created successfully.
